In [6]:
demand = 2400
plant1 = 1800
plant2 = 600

w1a = int(0.60*0.85*demand)
w1b = int(0.60*0.15*demand)
w2a = int(0.15*0.60*demand)
w2b = int(0.15*0.40*demand)
w3a = int(0.25*0.80*demand)
w3b = int(0.25*0.20*demand)

In [21]:
[w1a,w1b,w2a,w2b,w3a,w3b]

[1224, 216, 216, 144, 480, 120]

### equality constraints

    x10 + x20 = w1a
    x11 + x21 = w1b
    x12 + x22 = w2a
    x13 + x23 = w2b
    x14 + x24 = w3a
    x15 + x25 = w3b

    x10 + x11 + x12 + x13 + x14 + x15 = 1800
    x20 + x21 + x22 + x23 + x24 + x25 = 600

### inequality constraints

    None

### bounds

    ok, default = (0,None)
    
### objective function

    x10*0.80 + x11*1.20 + x12*1.50 + x13*1.60 + x14*1.90 + x15*2.10
    + x20*1.30 + x21*1.90 + x22*1.05 + x23*0.80 + x24*1.50 + x25*1.70

In [24]:
import numpy as np

LA_eq = []
for x in range(6):
    z = []
    for y in range(12):
        if y%6 == x: z.append(1)
        else:        z.append(0)
    LA_eq.append(z)    
LA_eq.append([1]*6+[0]*6)
LA_eq.append([0]*6+[1]*6)

A_eq = np.array(LA_eq)
b_eq = np.array([w1a,w1b,w2a,w2b,w3a,w3b,1800,600])

A_eq,b_eq

(array([[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]]),
 array([1224,  216,  216,  144,  480,  120, 1800,  600]))

In [25]:
from scipy import optimize

c = np.array([0.8,1.2,1.5,1.6,1.9,2.1, 1.3,1.9,1.05,0.8,1.5,1.7])

optimize.linprog(c,None,None,A_eq,b_eq)

     fun: 2648.4000000000005
 message: 'Optimization terminated successfully.'
     nit: 11
   slack: array([], dtype=float64)
  status: 0
 success: True
       x: array([ 1224.,   216.,     0.,     0.,   360.,     0.,     0.,     0.,
         216.,   144.,   120.,   120.])

In [26]:
c = np.array([0.8,1.2,1.5,1.6,1.9,2.1, 1.9,2.9,1.2,1.6,0.9,0.8])

optimize.linprog(c,None,None,A_eq,b_eq)

     fun: 2320.8000000000002
 message: 'Optimization terminated successfully.'
     nit: 9
   slack: array([], dtype=float64)
  status: 0
 success: True
       x: array([ 1224.,   216.,   216.,   144.,     0.,     0.,     0.,     0.,
           0.,     0.,   480.,   120.])

So if we look only at the returnables for the 1e year, the warehouse in the south is preferable.

### equality constraints

    x10 + x20 + x30 = w1a
    x11 + x21 + x31 = w1b
    x12 + x22 + x32 = w2a
    x13 + x23 + x33 = w2b
    x14 + x24 + x34 = w3a
    x15 + x25 + x35 = w3b

    x10 + x11 + x12 + x13 + x14 + x15 = 1800
    x20 + x21 + x22 + x23 + x24 + x25 + x30 + x31 + x32 + x33 + x34 + x35 = 600
    
    x20 + x21 + x22 + x23 + x24 + x25 - p1*600 = 0
    x30 + x31 + x32 + x33 + x34 + x35 - p2*600 = 0
    
    p1 + p2 = 1
    

### inequality constraints

    None

### bounds

    ok, default = (0,None)
    p1,p2 binary
    
### objective function

    x10*0.80 + x11*1.20 + x12*1.50 + x13*1.60 + x14*1.90 + x15*2.10
    + x20*1.30 + x21*1.90 + x22*1.05 + x23*0.80 + x24*1.50 + x25*1.70

In [28]:
from pulp import *

In [45]:
variables = ['x'+''.join(map(str,divmod(x,6))) for x in range(18)]
plants = ['p'+str(x) for x in range(2)]

LP_variables = [LpVariable(x, lowBound = 0, cat='Integer') for x in variables]
LP_variables += [LpVariable(x, lowBound = 0, cat='Binary') for x in plants]
print(LP_variables)

[x00, x01, x02, x03, x04, x05, x10, x11, x12, x13, x14, x15, x20, x21, x22, x23, x24, x25, p0, p1]


In [94]:
# consumption per warehouse
LP_expressions = [LpAffineExpression([(LP_variables[x],1),
                                      (LP_variables[x+6],1),
                                      (LP_variables[x+12],1)]) for x in range(6)]
# plants
LP_expressions += [LpAffineExpression([(LP_variables[-2],1),
                                        (LP_variables[-1],1)])]

# production per plant
LP_expressions += [LpAffineExpression([(LP_variables[x],1) for x in range(6)])]
LP_expressions += [LpAffineExpression([(LP_variables[x],1) for x in range(6,18)])]

LP_expressions += [LpAffineExpression([(LP_variables[x+y*6],1) for x in range(6)]
                                      +[(LP_variables[y-3],-600)]
                                     ) for y in range(1,3)]

In [95]:
rhs = [w1a,w1b,w2a,w2b,w3a,w3b,1,1800,600,0,0]

In [96]:
LP_constraints = [LpConstraint(x,sense=0,rhs=rhs[ix]) for ix,x in enumerate(LP_expressions)]

In [103]:
# objective function
costs = [0.8,1.2,1.5,1.6,1.9,2.1, 1.3,1.9,1.05,0.8,1.5,1.7, 1.9,2.9,1.2,1.6,0.9,0.8]

LP_function = LpAffineExpression([(x,y) for x,y in zip(LP_variables,costs)])

In [112]:
prob = LpProblem("The_Plant_Problem",LpMinimize)
prob += LP_function

for x in LP_constraints:
    prob += x
prob

The_Plant_Problem:
MINIMIZE
0.8*x00 + 1.2*x01 + 1.5*x02 + 1.6*x03 + 1.9*x04 + 2.1*x05 + 1.3*x10 + 1.9*x11 + 1.05*x12 + 0.8*x13 + 1.5*x14 + 1.7*x15 + 1.9*x20 + 2.9*x21 + 1.2*x22 + 1.6*x23 + 0.9*x24 + 0.8*x25 + 0
SUBJECT TO
_C1: x00 + x10 + x20 = 1224

_C2: x01 + x11 + x21 = 216

_C3: x02 + x12 + x22 = 216

_C4: x03 + x13 + x23 = 144

_C5: x04 + x14 + x24 = 480

_C6: x05 + x15 + x25 = 120

_C7: p0 + p1 = 1

_C8: x00 + x01 + x02 + x03 + x04 + x05 = 1800

_C9: x10 + x11 + x12 + x13 + x14 + x15 + x20 + x21 + x22 + x23 + x24 + x25
 = 600

_C10: - 600 p0 + x10 + x11 + x12 + x13 + x14 + x15 = 0

_C11: - 600 p1 + x20 + x21 + x22 + x23 + x24 + x25 = 0

VARIABLES
0 <= p0 <= 1 Integer
0 <= p1 <= 1 Integer
0 <= x00 Integer
0 <= x01 Integer
0 <= x02 Integer
0 <= x03 Integer
0 <= x04 Integer
0 <= x05 Integer
0 <= x10 Integer
0 <= x11 Integer
0 <= x12 Integer
0 <= x13 Integer
0 <= x14 Integer
0 <= x15 Integer
0 <= x20 Integer
0 <= x21 Integer
0 <= x22 Integer
0 <= x23 Integer
0 <= x24 Integer
0 <= x25

In [113]:
prob.writeLP("The_Plant_Problem.lp")
prob.solve()

1

In [118]:
for v in prob.variables():
    if v.varValue:
        print(v.name, "=", int(v.varValue))
        
print("Total Cost = ", value(prob.objective))

p1 = 1
x00 = 1224
x01 = 216
x02 = 216
x03 = 144
x24 = 480
x25 = 120
Total Cost =  2320.8
